https://www.kaggle.com/datasets/nasa/solar-eclipses?select=solar.csv

In [77]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [78]:
df=pd.read_csv('solar.csv')

In [79]:
df.head()

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Lunation Number,Saros Number,Eclipse Type,Gamma,Eclipse Magnitude,Latitude,Longitude,Sun Altitude,Sun Azimuth,Path Width (km),Central Duration
0,1,-1999 June 12,03:14:51,46438,-49456,5,T,-0.2701,1.0733,6.0N,33.3W,74,344,247,06m37s
1,2,-1999 December 5,23:45:23,46426,-49450,10,A,-0.2317,0.9382,32.9S,10.8E,76,21,236,06m44s
2,3,-1998 June 1,18:09:16,46415,-49444,15,T,0.4994,1.0284,46.2N,83.4E,60,151,111,02m15s
3,4,-1998 November 25,05:57:03,46403,-49438,20,A,-0.9045,0.9806,67.8S,143.8W,25,74,162,01m14s
4,5,-1997 April 22,13:19:56,46393,-49433,-13,P,-1.4670,0.1611,60.6S,106.4W,0,281,NaN,NaN


In [80]:
df.tail()

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Lunation Number,Saros Number,Eclipse Type,Gamma,Eclipse Magnitude,Latitude,Longitude,Sun Altitude,Sun Azimuth,Path Width (km),Central Duration
11893,11894,2998 December 10,03:18:31,4414,12355,187,P,1.2838,0.4773,67.2N,145.0E,0,179,NaN,NaN
11894,11895,2999 May 6,23:23:57,4417,12360,154,T,0.8388,1.0566,71.5N,177.3E,33,146,345,03m25s
11895,11896,2999 October 30,09:34:33,4420,12366,159,A-,-1.0023,0.9586,70.9S,84.7W,0,137,-,-
11896,11897,3000 April 26,14:18:06,4424,12372,164,T,0.1310,1.0222,21.1N,18.4W,82,166,76,02m11s
11897,11898,3000 October 19,16:10:16,4428,12378,169,H,-0.2303,1.0049,23.1S,51.6W,77,16,17,00m29s


In [81]:
df.columns

Index(['Catalog Number', 'Calendar Date', 'Eclipse Time', 'Delta T (s)',
       'Lunation Number', 'Saros Number', 'Eclipse Type', 'Gamma',
       'Eclipse Magnitude', 'Latitude', 'Longitude', 'Sun Altitude',
       'Sun Azimuth', 'Path Width (km)', 'Central Duration'],
      dtype='object')

In [82]:
df["Calendar Date"] = pd.to_datetime(df["Calendar Date"], errors="coerce")

In [83]:
event = df[df["Calendar Date"] == "2026-02-17"]
print(event.T)

                                  9564
Catalog Number                    9565
Calendar Date      2026-02-17 00:00:00
Eclipse Time                  12:13:06
Delta T (s)                         75
Lunation Number                    323
Saros Number                       121
Eclipse Type                         A
Gamma                          -0.9743
Eclipse Magnitude                0.963
Latitude                         64.7S
Longitude                        86.8E
Sun Altitude                        12
Sun Azimuth                        268
Path Width (km)                    616
Central Duration                02m20s


<b> RQ1: How did Gamma determine geometrical alignment?

In [84]:
gamma_value = event["Gamma"].values[0]
abs_gamma = abs(gamma_value)

print("Gamma:", gamma_value)
print("Absolute Gamma:", abs_gamma)

Gamma: -0.9743
Absolute Gamma: 0.9743


<b> RQ2: Did Magnitude, Path Width & Duration describe intensity?

In [87]:
features = ["Eclipse Magnitude", "Path Width (km)", "Central Duration"]
print(event[features])

      Eclipse Magnitude Path Width (km) Central Duration
9564              0.963             616           02m20s


In [88]:
df["Path Width (km)"] = pd.to_numeric(df["Path Width (km)"], errors="coerce")

In [89]:
import re
import numpy as np

def convert_duration(x):
    if pd.isna(x):
        return np.nan
    mins = re.search(r'(\d+)m', str(x))
    secs = re.search(r'(\d+)s', str(x))
    minutes = int(mins.group(1)) if mins else 0
    seconds = int(secs.group(1)) if secs else 0
    return minutes * 60 + seconds

df["Central Duration"] = df["Central Duration"].apply(convert_duration)

In [90]:
print(df[features].corr())

                   Eclipse Magnitude  Path Width (km)  Central Duration
Eclipse Magnitude           1.000000        -0.260606         -0.373988
Path Width (km)            -0.260606         1.000000          0.549954
Central Duration           -0.373988         0.549954          1.000000


<b> RQ3: When is the Next Eclipse & What Type?

In [91]:
future_eclipses = df[df["Calendar Date"] > "2026-02-17"].sort_values("Calendar Date")

next_eclipse = future_eclipses.iloc[0]

print("Next Eclipse Date:", next_eclipse["Calendar Date"])
print("Type:", next_eclipse["Eclipse Type"])
print("Magnitude:", next_eclipse["Eclipse Magnitude"])
print("Gamma:", next_eclipse["Gamma"])

Next Eclipse Date: 2026-02-22 00:00:00
Type: P
Magnitude: 0.3393
Gamma: -1.3595


In [92]:
if abs(next_eclipse["Gamma"]) < abs_gamma:
    print("Next eclipse is geometrically more central.")
else:
    print("Next eclipse is more peripheral.")

Next eclipse is more peripheral.


In [93]:
if next_eclipse["Eclipse Magnitude"] > event["Eclipse Magnitude"].values[0]:
    print("Next eclipse likely stronger.")
else:
    print("Next eclipse likely weaker.")

Next eclipse likely weaker.
